In [1]:
!pip install requests tensorflow python-dotenv

  Using cached tensorflow-2.16.1-cp311-cp311-win_amd64.whl.metadata (3.5 kB)
  Using cached tensorflow_intel-2.16.1-cp311-cp311-win_amd64.whl.metadata (5.0 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.5.4-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached h5py-3.11.0-cp311-cp311-win_amd64.whl.metadata (2.5 kB)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached ml_dtypes-0.3.2-cp311-cp311-win_amd64.whl.metadata (20 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached tensorboard-2.16.2-py3-none-any.whl.metadata (1.6 kB)
  Using cached keras-3.3.3-py3-none-any.whl.metadata (5.7 kB)
  Using ca

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
url = "https://docs.google.com/spreadsheets/d/1efoqCdfc-vLt9RvuNh2C6PpOUAb9Jbsr/edit?usp=sharing&ouid=102816388336503089633&rtpof=true&sd=true"
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
users = pd.read_excel(path, sheet_name='User')
products = pd.read_excel(path, sheet_name='Product')
interactions = pd.read_excel(path, sheet_name='New_Interaction')

In [ ]:
users.head()

,user_id,user_name,age,gender,location
0,1,Erin,18,F,Bali
1,2,Faye,17,F,Bogor
2,3,Cora,17,M,Jogjakarta
3,4,Olive,25,M,Jember
4,5,Jack,18,F,Jogjakarta


In [ ]:
products.head()

,product_id,product_name,category,price,location
0,101,Damaindah Meja Belajar Kayu Set Kursi / Meja B...,Meja,155000.0,Malang
1,102,Homedoki Meja / Meja Makan / Meja Komputer / M...,Meja,124000.0,Semarang
2,103,Sakula Meja kantor meja kerja Meja Komputer Pe...,Meja,107000.0,Palembang
3,104,Meja Portable Stand Laptop Meja Laptop Standin...,Meja,99500.0,Malang
4,105,PiPi Furniture Meja Gaming / Meja komputer / M...,Meja,446000.0,Palembang


In [ ]:
interactions.head()

,user_id,product_id,interaction_type,interaction_value,timestamp
0,21,131,add_to_cart,1,2024-06-05 07:46:05.923
1,238,1217,purchase,1,2024-06-05 07:46:05.923
2,212,827,view,2,2024-06-05 07:46:05.923
3,13,1183,purchase,1,2024-06-05 07:46:05.923
4,9,1063,purchase,1,2024-06-05 07:46:05.923


In [ ]:
# Encode user_id and product_id
user_ids = interactions['user_id'].unique().tolist()
product_ids = interactions['product_id'].unique().tolist()

user2user_encoded = {x: i for i, x in enumerate(user_ids)}
product2product_encoded = {x: i for i, x in enumerate(product_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
productencoded2product = {i: x for i, x in enumerate(product_ids)}

interactions['user'] = interactions['user_id'].map(user2user_encoded)
interactions['product'] = interactions['product_id'].map(product2product_encoded)

# Split the data
train, test = train_test_split(interactions, test_size=0.2, random_state=42)

# Convert data into required format
x_train = [train['user'].values, train['product'].values]
y_train = train['interaction_value'].values
x_test = [test['user'].values, test['product'].values]
y_test = test['interaction_value'].values

In [ ]:
# Define the model
user_input = Input(shape=(1,))
user_embedding = Embedding(len(user_ids), 50, input_length=1)(user_input)
user_vec = Flatten()(user_embedding)

product_input = Input(shape=(1,))
product_embedding = Embedding(len(product_ids), 50, input_length=1)(product_input)
product_vec = Flatten()(product_embedding)

dot_product = Dot(axes=1)([user_vec, product_vec])

model = Model([user_input, product_input], dot_product)
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(x_train, y_train, epochs=20, verbose=1, validation_split=0.2)

Epoch 1/20
40/40 [==============================] - 1s 6ms/step - loss: 4.3006 - val_loss: 4.4778
Epoch 2/20
40/40 [==============================] - 0s 3ms/step - loss: 4.2651 - val_loss: 4.4776
Epoch 3/20
40/40 [==============================] - 0s 3ms/step - loss: 4.2235 - val_loss: 4.4770
Epoch 4/20
40/40 [==============================] - 0s 3ms/step - loss: 4.1670 - val_loss: 4.4762
Epoch 5/20
40/40 [==============================] - 0s 3ms/step - loss: 4.0881 - val_loss: 4.4751
Epoch 6/20
40/40 [==============================] - 0s 3ms/step - loss: 3.9827 - val_loss: 4.4733
Epoch 7/20
40/40 [==============================] - 0s 3ms/step - loss: 3.8439 - val_loss: 4.4713
Epoch 8/20
40/40 [==============================] - 0s 3ms/step - loss: 3.6742 - val_loss: 4.4689
Epoch 9/20
40/40 [==============================] - 0s 3ms/step - loss: 3.4757 - val_loss: 4.4652
Epoch 10/20
40/40 [==============================] - 0s 3ms/step - loss: 3.2492 - val_loss: 4.4617
Epoch 11/20
40/40 [

In [ ]:
# Function to get recommendations for a specific user
def recommend_products(user_id, model, interactions, user2user_encoded, product2product_encoded, productencoded2product, products, top_n=20):
    user_encoded = user2user_encoded[user_id]
    product_ids = list(product2product_encoded.values())
    user_product_array = np.hstack(
        ([[user_encoded]] * len(product_ids), np.array(product_ids).reshape(-1, 1))
    )

    predictions = model.predict([user_product_array[:, 0], user_product_array[:, 1]])
    predictions = predictions.flatten()

    top_indices = predictions.argsort()[-top_n:][::-1]
    recommended_product_ids = [productencoded2product[x] for x in top_indices]

    recommended_products = products[products['product_id'].isin(recommended_product_ids)]
    return recommended_products

# Example: Get recommendations for user with user_id 1
user_id = 23
recommended_products = recommend_products(user_id, model, interactions, user2user_encoded, product2product_encoded, productencoded2product, products)
print(f"Recommended products for user {user_id}:")
print(recommended_products)

33/33 [==============================] - 0s 5ms/step
Recommended products for user 23:
      product_id                                       product_name  \
95           196  MEJA TULIS/ MEJA KECIL/ MEJA BELAJAR MINIMALIS...   
102          203  Meja Laptop Portable Meja Laptop Roda Meja Lip...   
141          242  Meja Belajar Aesthetic Minimalis Serbaguna Mej...   
195          296           meja makan murah 80x40x75 (NON FINISING)   
300          401  PROMO !!! 85rb Dapat 1pc Kasur Tebal 6cm MUAT ...   
373          474  PROMO KASUR BUSA SUPER ANTI KEMPES 180X140X10 ...   
380          481         Kasur Gender Bulu Rasfur 200 x 200 x 10 cm   
392          493  Free 2 BANTAL Kasur Gender/ Cimol bulu empuk 1...   
670          771  DAP D-B01 Portable Mini Hand Fan - Kipas Angin...   
708          809  LOVE Lemari Kain Portable lemari 2 Sisi Lemari...   
811          912  Rel Lemari / Rel Timbul SD 380 Pintu Lemari Ge...   
825          926  LEMARI RAK SERBAGUNA SUSUN 3 4 5 KAYU DENGA